In [1]:
!pip install Kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
!chmod 600 /root/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [4]:
!kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 398, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [5]:
!kaggle datasets download -d vivmankar/asian-vs-african-elephant-image-classification

Dataset URL: https://www.kaggle.com/datasets/vivmankar/asian-vs-african-elephant-image-classification
License(s): CC0-1.0
 93% 48.0M/51.8M [00:02<00:00, 34.8MB/s]
100% 51.8M/51.8M [00:02<00:00, 25.2MB/s]


In [6]:
!unzip asian-vs-african-elephant-image-classification.zip

Archive:  asian-vs-african-elephant-image-classification.zip
  inflating: dataset/test/African/af_te1.jpg  
  inflating: dataset/test/African/af_te10.jpg  
  inflating: dataset/test/African/af_te11.jpg  
  inflating: dataset/test/African/af_te12.jpg  
  inflating: dataset/test/African/af_te13.jpg  
  inflating: dataset/test/African/af_te14.jpg  
  inflating: dataset/test/African/af_te15.jpg  
  inflating: dataset/test/African/af_te16.jpg  
  inflating: dataset/test/African/af_te17.jpg  
  inflating: dataset/test/African/af_te18.jpg  
  inflating: dataset/test/African/af_te19.jpg  
  inflating: dataset/test/African/af_te2.jpg  
  inflating: dataset/test/African/af_te20.jpg  
  inflating: dataset/test/African/af_te21.jpg  
  inflating: dataset/test/African/af_te22.jpg  
  inflating: dataset/test/African/af_te23.jpg  
  inflating: dataset/test/African/af_te24.jpg  
  inflating: dataset/test/African/af_te25.jpg  
  inflating: dataset/test/African/af_te26.jpg  
  inflating: dataset/test/Afr

In [7]:
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader
from tqdm import tqdm  # For nice progress bar!

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.flat = nn.Flatten(-3, -1)
        self.fc1 = nn.Linear(43808, 4500)
        self.act2 = nn.ReLU()
        self.fc2 = nn.Linear(4500, 1)
        self.act3 = nn.Sigmoid()
    def forward(self, x):
        x = self.act1(self.conv1(x))
        #print("Conv1:", x.shape)
        x = self.pool1(x)
        #print("Pooling1:", x.shape)
        x = self.flat(x)
        #print("Flattened:", x.shape)
        x = self.fc1(x)
        x = self.act1(x)
        #print("FC1:", x.shape)
        x = self.fc2(x)
        #x = self.act3(x)
        #print("Output:", x.shape)
        return x

Setting the device

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Initialize the network

In [10]:
model = CNN().to(device)

Hyper-Parameters

In [11]:
in_channels = 3
num_classes = 2
learning_rate = 3e-4 # karpathy's constant
batch_size = 4
num_epochs = 3

Load the Data

In [12]:
trans_train =  transforms.Compose([transforms.Resize(75),
                                       transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(75),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])

trans_valid = transforms.Compose([transforms.Resize(75),
                                      transforms.CenterCrop(75),
                                      transforms.ToTensor()])

data_dir = 'dataset'

train_dataset = datasets.ImageFolder(data_dir + '/train',
                                  transform=trans_train)
test_dataset = datasets.ImageFolder(data_dir + '/test',
                                 transform=trans_valid)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


Loss and optimizer

In [13]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Train Network

In [14]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        #print(scores.shape, targets.shape)
        loss = criterion(scores, targets.reshape(-1,1).float())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples




100%|██████████| 210/210 [00:14<00:00, 14.05it/s]


In [15]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 50.00
Accuracy on test set: 51.60


In [21]:
!git config --global user.email "kunalbitey1001@gmail.com"
!git config --global user.name "Kunal"

In [23]:
!echo "# Elephant_Classification" >> README.md
!git init
!git add README.md
!git add .
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/boaluma/Elephant_Classification.git
!git push -u origin main

Reinitialized existing Git repository in /content/.git/
[main 4c12430] first commit
 1 file changed, 1 insertion(+)
error: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address


In [22]:
!git remote add origin https://github.com/boaluma/Elephant_Classification.git
!git branch -M main
!git push -u origin main

error: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address
